#### 1. Déclaration des packages NuGet

In [54]:
// Déclaration des packages NuGet requis
#r "nuget: CsvHelper, 27.1.1"
#r "nuget: ClosedXML, 0.95.4"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: Lucene.Net.Analysis.Common, 4.8.0-beta00013"


Installed Packages ClosedXML, 0.95.4 CsvHelper, 27.1.1 Lucene.Net.Analysis.Common, 4.8.0-beta00013 XPlot.Plotly.Interactive, 4.1.0

#### 2. Définition des paramètres

Il s'agit du coefficient et du nom du prof, d'éventuels paramètres de sécurité ou d'affichage.

In [55]:
// Déclaration des variables pour la pondération et d'autres paramètres

int numberOfProjects = 1; 
int[] nbEvalFieldsPerProject = { 3, 3, 3 };

decimal teacherWeight = 1m;
string professorEmail = "jsboige@gmail.com";
// Ajouter d'autres paramètres selon les besoins


#### 3. Upload et Chargement des Inscriptions

In [56]:
using Microsoft.DotNet.Interactive;
using System.IO;
using CsvHelper;
using System.Globalization;
using CsvHelper.Configuration;

public class StudentRecord
{
    public string Nom { get; set; }
    public string Prénom { get; set; }
    public string Classe { get; set; }
    
    public string Sujet { get; set; }
    public List<string> Sujets => new(){Sujet};
    public List<decimal> Notes { get; set; } = new();
    public decimal Moyenne => Notes.Any()? Notes.Average(): 0;

}



public class StudentMap : ClassMap<StudentRecord>
{
    public StudentMap()
    {
        Map(m => m.Nom).Name("Nom");
        Map(m => m.Prénom).Name("Prénom");
        // Map(m => m.Classe).Name("Classe (CV/NLP)");
        Map(m => m.Sujet).Name("Sujet IA Symbolique");
    }
}

// Upload et lecture du fichier CSV des inscriptions
var studentFileInput = await Kernel.GetInputAsync(
    "Veuillez uploader le fichier CSV contenant les inscriptions des étudiants:",
    typeHint: "file");

display($"Fichier sélectionné : {studentFileInput}");

List<StudentRecord> studentRecords;
using (var studentReader = new StreamReader(studentFileInput))
{
    using (var studentCsv = new CsvReader(studentReader, CultureInfo.InvariantCulture))
    {
        studentCsv.Context.RegisterClassMap<StudentMap>();
        studentRecords = studentCsv.GetRecords<StudentRecord>().ToList();
    }
}

// Affichage des 5 premières lignes pour vérification
display(studentRecords.Take(5), "application/json");


// Affichage des élèves en anomalie (non inscrits)


Fichier sélectionné : g:\Mon Drive\MyIA\Formation\Epita\2024\2024 - SCIA - NLP - IA Symbolique - Inscription.xlsx - eleves.csv

#### 4. Chargement des fichiers de notes

Dans une boucle, on charge les différents exports csv des fichiers produits par Google Forms.

In [57]:
using System.Collections.Generic;
using CsvHelper.Configuration;
using System.Globalization;
using System.IO;
using System.Linq;
using Microsoft.DotNet.Interactive.Formatting;


public class EvaluationRecord
{
    public DateTime Date { get; set; }
    public string Email { get; set; }
    public string Nom { get; set; }
    public string Prénom { get; set; }
    public string Groupe { get; set; }
    public string SujetLibre { get; set; }
    public int NoteCommunication { get; set; }
    public int NoteThéorique { get; set; }
    public int NoteTechnique { get; set; }
    public int NoteOrganisation { get; set; }
    public string PointsPositifs { get; set; }
    public string PointsNégatifs { get; set; }
    public string Recommandations { get; set; }

    public decimal Note => ((decimal) (NoteCommunication + NoteTechnique + NoteThéorique + NoteOrganisation) * 2)  / (decimal) NbEvalFields;

    public bool IsTeacher { get; set; } 
    public int NbEvalFields { get; set; } = 4;

}




public class EvaluationMap : ClassMap<EvaluationRecord>
{
    public EvaluationMap()
    {
        Map(m => m.Date).Name("Horodateur");
        Map(m => m.Email).Name("Adresse e-mail");
        Map(m => m.Nom).Name("Votre nom");
        Map(m => m.Prénom).Name("Votre prénom");
        Map(m => m.Groupe).Name("Groupe à évaluer");
        Map(m => m.SujetLibre).Name("Sujet de la présentation");
        Map(m => m.NoteCommunication).Name("Qualité de la présentation (communication, la forme)");
        Map(m => m.NoteThéorique).Name("Qualité théorique (principes utilisés, classe d'algorithmes, contexte et explications des performances et des problèmes, histoire etc.)");
        Map(m => m.NoteTechnique).Name("Qualité technique (livrables, commits, qualité du code, démos, résultats, perspectives)");
        // Map(m => m.NoteOrganisation).Name("Organisation (planning, répartition des tâches, collaboration, intégration au projet Github)");
        Map(m => m.PointsPositifs).Name("Points positifs de la présentation");
        Map(m => m.PointsNégatifs).Name("Points négatifs de la présentation");
        Map(m => m.Recommandations).Name("Recommandations pour s'améliorer");
    }
}
var evaluations = new List<List<EvaluationRecord>>();
for (int i = 0; i < numberOfProjects; i++)
{
    var fileInput = await Kernel.GetInputAsync($"Veuillez uploader le fichier CSV pour le projet {i + 1}:", typeHint: "file");
    using var evalReader = new StreamReader(fileInput);
    var csvConfig = new CsvConfiguration(CultureInfo.CurrentCulture)
    {
        Delimiter = ",",
        HasHeaderRecord = true,
        MissingFieldFound = null,
        BadDataFound = null,
        IgnoreBlankLines = true,
        TrimOptions = TrimOptions.Trim,
    };
    using var evalCsv = new CsvReader(evalReader, csvConfig);
    evalCsv.Context.RegisterClassMap<EvaluationMap>();
    var projectEvaluations = evalCsv.GetRecords<EvaluationRecord>().ToList();

    // Validation du sujet libre contre le groupe évalué (simplifié ici)
    foreach (var eval in projectEvaluations)
    {
        eval.NbEvalFields = nbEvalFieldsPerProject[i];
        if (eval.Email == professorEmail)
        {
            eval.IsTeacher = true;
        }
        
        var expectedSubject = studentRecords.FirstOrDefault(s => s.Sujet == eval.Groupe)?.Sujet;
        if (expectedSubject != null && !eval.Groupe.Contains(expectedSubject))
        {
            display($"Attention: Le sujet '{eval.Groupe}' ne correspond pas au groupe '{eval.Groupe}' attendu pour '{expectedSubject}'.");
        }
    }

    // Afficher les données chargées pour vérification
    display($"Evaluations du projet {System.IO.Path.GetFileName(fileInput)}");
     projectEvaluations.Take(5).ToList().ToDisplayString("text/html").DisplayAs("text/markdown");
    evaluations.Add(projectEvaluations);
}





Evaluations du projet Epita - 2024 - SCIA - NLP - Evaluations IASY (réponses) - Réponses au formulaire 1.csv

<table><thead><tr><th><i>index</i></th><th>value</th></tr></thead><tbody><tr><td>0</td><td><details class="dni-treeview"><summary><span class="dni-code-hint"><code>Submission#58+EvaluationRecord</code></span></summary><div><table><thead><tr></tr></thead><tbody><tr><td>Date</td><td><span>2024-06-12 14:15:49Z</span></td></tr><tr><td>Email</td><td><div class="dni-plaintext"><pre>loris.lin@epita.fr</pre></div></td></tr><tr><td>Nom</td><td><div class="dni-plaintext"><pre>Lin</pre></div></td></tr><tr><td>Prénom</td><td><div class="dni-plaintext"><pre>Loris</pre></div></td></tr><tr><td>Groupe</td><td><div class="dni-plaintext"><pre>Les locuteurs pragmatiques et le discours rationnel</pre></div></td></tr><tr><td>SujetLibre</td><td><div class="dni-plaintext"><pre>Les locuteurs pragmatiques et le discours rationnel</pre></div></td></tr><tr><td>NoteCommunication</td><td><div class="dni-plaintext"><pre>10</pre></div></td></tr><tr><td>NoteThéorique</td><td><div class="dni-plaintext"><pre>10</pre></div></td></tr><tr><td>NoteTechnique</td><td><div class="dni-plaintext"><pre>10</pre></div></td></tr><tr><td>NoteOrganisation</td><td><div class="dni-plaintext"><pre>0</pre></div></td></tr><tr><td>PointsPositifs</td><td><div class="dni-plaintext"><pre>Fluidit&#233; et des jolies slides</pre></div></td></tr><tr><td>PointsNégatifs</td><td><div class="dni-plaintext"><pre></pre></div></td></tr><tr><td>Recommandations</td><td><div class="dni-plaintext"><pre></pre></div></td></tr><tr><td>Note</td><td><div class="dni-plaintext"><pre>20</pre></div></td></tr><tr><td>IsTeacher</td><td><div class="dni-plaintext"><pre>False</pre></div></td></tr><tr><td>NbEvalFields</td><td><div class="dni-plaintext"><pre>3</pre></div></td></tr></tbody></table></div></details></td></tr><tr><td>1</td><td><details class="dni-treeview"><summary><span class="dni-code-hint"><code>Submission#58+EvaluationRecord</code></span></summary><div><table><thead><tr></tr></thead><tbody><tr><td>Date</td><td><span>2024-06-12 14:17:09Z</span></td></tr><tr><td>Email</td><td><div class="dni-plaintext"><pre>emon.barberis@epita.fr</pre></div></td></tr><tr><td>Nom</td><td><div class="dni-plaintext"><pre>Barberis</pre></div></td></tr><tr><td>Prénom</td><td><div class="dni-plaintext"><pre>Emon</pre></div></td></tr><tr><td>Groupe</td><td><div class="dni-plaintext"><pre>Les locuteurs pragmatiques et le discours rationnel</pre></div></td></tr><tr><td>SujetLibre</td><td><div class="dni-plaintext"><pre>Locuteurs pragmatiques et le discours rationnel</pre></div></td></tr><tr><td>NoteCommunication</td><td><div class="dni-plaintext"><pre>10</pre></div></td></tr><tr><td>NoteThéorique</td><td><div class="dni-plaintext"><pre>10</pre></div></td></tr><tr><td>NoteTechnique</td><td><div class="dni-plaintext"><pre>10</pre></div></td></tr><tr><td>NoteOrganisation</td><td><div class="dni-plaintext"><pre>0</pre></div></td></tr><tr><td>PointsPositifs</td><td><div class="dni-plaintext"><pre></pre></div></td></tr><tr><td>PointsNégatifs</td><td><div class="dni-plaintext"><pre></pre></div></td></tr><tr><td>Recommandations</td><td><div class="dni-plaintext"><pre></pre></div></td></tr><tr><td>Note</td><td><div class="dni-plaintext"><pre>20</pre></div></td></tr><tr><td>IsTeacher</td><td><div class="dni-plaintext"><pre>False</pre></div></td></tr><tr><td>NbEvalFields</td><td><div class="dni-plaintext"><pre>3</pre></div></td></tr></tbody></table></div></details></td></tr><tr><td>2</td><td><details class="dni-treeview"><summary><span class="dni-code-hint"><code>Submission#58+EvaluationRecord</code></span></summary><div><table><thead><tr></tr></thead><tbody><tr><td>Date</td><td><span>2024-06-12 14:19:57Z</span></td></tr><tr><td>Email</td><td><div class="dni-plaintext"><pre>lilian.schall@epita.fr</pre></div></td></tr><tr><td>Nom</td><td><div class="dni-plaintext"><pre>Schall</pre></div></td></tr><tr><td>Prénom</td><td><div class="dni-plaintext"><pre>Lilian</pre></div></td></tr><tr><td>Groupe</td><td><div class="dni-plaintext"><pre>Les locuteurs pragmatiques et le discours rationnel</pre></div></td></tr><tr><td>SujetLibre</td><td><div class="dni-plaintext"><pre>Les locuteurs pragmatiques et le discours rationnel</pre></div></td></tr><tr><td>NoteCommunication</td><td><div class="dni-plaintext"><pre>10</pre></div></td></tr><tr><td>NoteThéorique</td><td><div class="dni-plaintext"><pre>10</pre></div></td></tr><tr><td>NoteTechnique</td><td><div class="dni-plaintext"><pre>10</pre></div></td></tr><tr><td>NoteOrganisation</td><td><div class="dni-plaintext"><pre>0</pre></div></td></tr><tr><td>PointsPositifs</td><td><div class="dni-plaintext"><pre></pre></div></td></tr><tr><td>PointsNégatifs</td><td><div class="dni-plaintext"><pre></pre></div></td></tr><tr><td>Recommandations</td><td><div class="dni-plaintext"><pre></pre></div></td></tr><tr><td>Note</td><td><div class="dni-plaintext"><pre>20</pre></div></td></tr><tr><td>IsTeacher</td><td><div class="dni-plaintext"><pre>False</pre></div></td></tr><tr><td>NbEvalFields</td><td><div class="dni-plaintext"><pre>3</pre></div></td></tr></tbody></table></div></details></td></tr><tr><td>3</td><td><details class="dni-treeview"><summary><span class="dni-code-hint"><code>Submission#58+EvaluationRecord</code></span></summary><div><table><thead><tr></tr></thead><tbody><tr><td>Date</td><td><span>2024-06-12 14:28:18Z</span></td></tr><tr><td>Email</td><td><div class="dni-plaintext"><pre>lyes.benacer@epita.fr</pre></div></td></tr><tr><td>Nom</td><td><div class="dni-plaintext"><pre>Benacer</pre></div></td></tr><tr><td>Prénom</td><td><div class="dni-plaintext"><pre>Lyes</pre></div></td></tr><tr><td>Groupe</td><td><div class="dni-plaintext"><pre>Les locuteurs pragmatiques et le discours rationnel</pre></div></td></tr><tr><td>SujetLibre</td><td><div class="dni-plaintext"><pre>Locutuers pragmatiques</pre></div></td></tr><tr><td>NoteCommunication</td><td><div class="dni-plaintext"><pre>9</pre></div></td></tr><tr><td>NoteThéorique</td><td><div class="dni-plaintext"><pre>9</pre></div></td></tr><tr><td>NoteTechnique</td><td><div class="dni-plaintext"><pre>10</pre></div></td></tr><tr><td>NoteOrganisation</td><td><div class="dni-plaintext"><pre>0</pre></div></td></tr><tr><td>PointsPositifs</td><td><div class="dni-plaintext"><pre></pre></div></td></tr><tr><td>PointsNégatifs</td><td><div class="dni-plaintext"><pre></pre></div></td></tr><tr><td>Recommandations</td><td><div class="dni-plaintext"><pre></pre></div></td></tr><tr><td>Note</td><td><div class="dni-plaintext"><pre>18,666666666666666666666666667</pre></div></td></tr><tr><td>IsTeacher</td><td><div class="dni-plaintext"><pre>False</pre></div></td></tr><tr><td>NbEvalFields</td><td><div class="dni-plaintext"><pre>3</pre></div></td></tr></tbody></table></div></details></td></tr><tr><td>4</td><td><details class="dni-treeview"><summary><span class="dni-code-hint"><code>Submission#58+EvaluationRecord</code></span></summary><div><table><thead><tr></tr></thead><tbody><tr><td>Date</td><td><span>2024-06-12 14:28:34Z</span></td></tr><tr><td>Email</td><td><div class="dni-plaintext"><pre>romain.trefault@epita.fr</pre></div></td></tr><tr><td>Nom</td><td><div class="dni-plaintext"><pre>Trefault</pre></div></td></tr><tr><td>Prénom</td><td><div class="dni-plaintext"><pre>Romain</pre></div></td></tr><tr><td>Groupe</td><td><div class="dni-plaintext"><pre>Les locuteurs pragmatiques et le discours rationnel</pre></div></td></tr><tr><td>SujetLibre</td><td><div class="dni-plaintext"><pre>Les locuteurs pragmatiques et le discours rationnel</pre></div></td></tr><tr><td>NoteCommunication</td><td><div class="dni-plaintext"><pre>9</pre></div></td></tr><tr><td>NoteThéorique</td><td><div class="dni-plaintext"><pre>9</pre></div></td></tr><tr><td>NoteTechnique</td><td><div class="dni-plaintext"><pre>10</pre></div></td></tr><tr><td>NoteOrganisation</td><td><div class="dni-plaintext"><pre>0</pre></div></td></tr><tr><td>PointsPositifs</td><td><div class="dni-plaintext"><pre>Complet, code + partie th&#233;orique, bonne illustration et explication</pre></div></td></tr><tr><td>PointsNégatifs</td><td><div class="dni-plaintext"><pre>trop monotone</pre></div></td></tr><tr><td>Recommandations</td><td><div class="dni-plaintext"><pre></pre></div></td></tr><tr><td>Note</td><td><div class="dni-plaintext"><pre>18,666666666666666666666666667</pre></div></td></tr><tr><td>IsTeacher</td><td><div class="dni-plaintext"><pre>False</pre></div></td></tr><tr><td>NbEvalFields</td><td><div class="dni-plaintext"><pre>3</pre></div></td></tr></tbody></table></div></details></td></tr></tbody></table><style>
.dni-code-hint {
    font-style: italic;
    overflow: hidden;
    white-space: nowrap;
}
.dni-treeview {
    white-space: nowrap;
}
.dni-treeview td {
    vertical-align: top;
    text-align: start;
}
details.dni-treeview {
    padding-left: 1em;
}
table td {
    text-align: start;
}
table tr { 
    vertical-align: top; 
    margin: 0em 0px;
}
table tr td pre 
{ 
    vertical-align: top !important; 
    margin: 0em 0px !important;
} 
table th {
    text-align: start;
}
</style>

#### 5. Définition des structures de croisement et validation des données

On commence par définir les structures de données qui vont permettre de rapprocher évaluations et inscriptions

In [58]:

using Lucene.Net.Analysis;
using Lucene.Net.Analysis.Core;
using Lucene.Net.Analysis.Standard;
using Lucene.Net.Analysis.Miscellaneous;
using Lucene.Net.Analysis.TokenAttributes;
using Lucene.Net.Util;
using System.IO;





public class GroupEvaluation
{
    public decimal TeacherWeight { get; set; }
    public string Groupe { get; set; }
    
    public decimal Moyenne
    {
        get
        {
            var studentEvals = Evaluations.Where(e => !e.IsTeacher).ToList();
            var teacherEvals = Evaluations.Where(e => e.IsTeacher).ToList();
            decimal studentAvg = studentEvals.Any() ? studentEvals.Average(e => e.Note) : 0;
            decimal teacherAvg = teacherEvals.Any() ? teacherEvals.Average(e => e.Note) : 0;
            decimal toReturn = studentEvals.Any() ? (studentAvg + (teacherEvals.Any() ? teacherAvg * TeacherWeight : 0)) / (1 + (teacherEvals.Any() ? TeacherWeight : 0)) : 0;
            return toReturn;
        }
    }

    
     public decimal EcartType
    {
        get
        {
            var studentEvals = Evaluations.Where(e => !e.IsTeacher).ToList();
            var teacherEvals = Evaluations.Where(e => e.IsTeacher).ToList();

            if (!studentEvals.Any()) return 0;

            decimal moyenne = Moyenne;
            decimal varianceEtudiant = studentEvals.Any() ? (decimal) studentEvals.Select(e => Math.Pow((double)(e.Note - moyenne), 2)).Average(): 0;
            decimal varianceProfesseur = teacherEvals.Any() ? (decimal) Math.Pow((double)(teacherEvals.Average(e => e.Note) - moyenne), 2) : 0;

            var combinedVariance = (double) ((varianceEtudiant + varianceProfesseur * TeacherWeight) / (1 + TeacherWeight));
            return (decimal)Math.Sqrt(combinedVariance);
        }
    }

        
    public decimal NoteRectifiée { get; set; }

    public List<EvaluationRecord> Evaluations { get; set; }
    public DateTime Date =>  Evaluations.Skip(Evaluations.Count / 2).First().Date;
    public List<StudentRecord> GroupMembers { get; set; }   
}

public class ProjectEvaluation
{

    public string ProfessorEmail {get; set;}

    public class CustomAnalyzer : Analyzer
    {
       private LuceneVersion version;
    
       public CustomAnalyzer(LuceneVersion version)
       {
           this.version = version;
       }
    
       protected override TokenStreamComponents CreateComponents(string fieldName, TextReader reader)
       {
           var tokenizer = new StandardTokenizer(version, reader);
           TokenStream tokenStream = new LowerCaseFilter(version, tokenizer);
           tokenStream = new ASCIIFoldingFilter(tokenStream);
           return new TokenStreamComponents(tokenizer, tokenStream);
       }
    }

    public List<GroupEvaluation> GroupedEvaluations { get; set; } = new List<GroupEvaluation>();

    public decimal Moyenne => GroupedEvaluations.Select(n => n.Moyenne * n.GroupMembers.Count).Sum() / GroupedEvaluations.Select(n => n.GroupMembers.Count).Sum();

    public decimal EcartType => (decimal) Math.Sqrt(GroupedEvaluations.Select(n => Math.Pow((double) (n.Moyenne - Moyenne), 2) * n.GroupMembers.Count).Sum() 
    / GroupedEvaluations.Select(n => n.GroupMembers.Count).Sum());

    public static string FoldAccents(string text)
    {
        using (var analyzer = new CustomAnalyzer(LuceneVersion.LUCENE_48))
        using (var reader = new StringReader(text))
        using (var tokenStream = analyzer.GetTokenStream("", reader))
        {
            var charTermAttribute = tokenStream.AddAttribute<ICharTermAttribute>();
            tokenStream.Reset();
            var output = string.Empty;
            while (tokenStream.IncrementToken())
            {
                output += charTermAttribute.ToString() + " ";
            }
            tokenStream.End();
            return output.Trim();
        }
    }


    public string ProjectName(string name)
    {
        var firstName = name.Split(" ").First().Split("-").First();
        firstName = firstName.Substring(0, Math.Min(5, firstName.Length));
        return FoldAccents(firstName.ToLowerInvariant().Trim());
    }

    public void FilterEvaluations()
    {
        // display($"Noms des inscrits : ");
        // foreach (var groupEval in GroupedEvaluations)
        // {
        //     foreach (var student in groupEval.GroupMembers)
        //     {
        //         display($"{student.Prénom} ({ProjectName(student.Prénom)}) {student.Nom} ({ProjectName(student.Nom)})");
        //     }
        // }
        foreach (var groupEval in GroupedEvaluations)
        {
            groupEval.Evaluations = groupEval.Evaluations.Where(e => this.IsValidEvaluation(groupEval, e)).ToList();
        }
    }


    public bool MatchNames(StudentRecord student, EvaluationRecord eval)
    {
        return (ProjectName(student.Nom) == ProjectName(eval.Nom) && ProjectName(student.Prénom) == ProjectName(eval.Prénom))
        || (ProjectName(student.Prénom) == ProjectName(eval.Nom) && ProjectName(student.Nom) == ProjectName(eval.Prénom));
    }

    public bool MatchNames(EvaluationRecord eval1, EvaluationRecord eval2)
    {
        return (ProjectName(eval1.Nom) == ProjectName(eval2.Nom) && ProjectName(eval1.Prénom) == ProjectName(eval2.Prénom))
        || (ProjectName(eval1.Prénom) == ProjectName(eval2.Nom) && ProjectName(eval1.Nom) == ProjectName(eval2.Prénom));
    }

    public bool IsValidEvaluation(GroupEvaluation groupEvaluation, EvaluationRecord eval)
    {
       // Vérifier que la note n'est pas attribuée avant la présentation
       // et que les notes ne sont pas extrêmement hautes ou basses sans justification
       if (eval.Note < 1 && eval.Note > 19) {
              display($"Evaluation écartée: {eval}, Note invalide ({eval.Note})");
            return false;
       }

       // Vérifier que la date est cohérente avec les autres évaluations du groupe
       if (groupEvaluation.Date >= eval.Date + TimeSpan.FromHours(5)){
              display($"Evaluation écartée: {eval}, Etudiant {eval.Prénom} ({this.ProjectName(eval.Prénom)}) {eval.Nom} ({this.ProjectName(eval.Nom)}) a évalué trop tôt ({eval.Date})");
              return false;
       } 
       if (groupEvaluation.Date < eval.Date - TimeSpan.FromHours(2)) {
                display($"Evaluation écartée: {eval}, Etudiant {eval.Prénom} ({this.ProjectName(eval.Prénom)}) {eval.Nom} ({this.ProjectName(eval.Nom)}) a évalué trop tard ({eval.Date})");
                return false;
        }

       // L'étudiant doit être inscrit.
       if (eval.Email != ProfessorEmail && !this.GroupedEvaluations.Exists(s => s.GroupMembers.Exists(s => MatchNames(s, eval)))) {
                           display($"Evaluation écartée: {eval}, Etudiant {eval.Prénom} ({this.ProjectName(eval.Prénom)}) {eval.Nom} ({this.ProjectName(eval.Nom)}) non inscrit ({eval.Email})");
                           
                           return false;
                       }


       // Interdiction de voter pour soi-même
       if(groupEvaluation.GroupMembers.Exists(s => MatchNames(s, eval))) {
                           display($"Evaluation écartée: {eval}, Etudiant {eval.Prénom} ({this.ProjectName(eval.Prénom)}) {eval.Nom} ({this.ProjectName(eval.Nom)}) membre du groupe évalué, ");
                           return false;
                       }

       // Interdiction aux doublons
       if (groupEvaluation.Evaluations.Count(e => MatchNames(e, eval)) > 1) {
                           display($"Evaluation écartée: {eval}, {eval.Prénom} ({this.ProjectName(eval.Prénom)}) {eval.Nom} ({this.ProjectName(eval.Nom)}) a évalué plusieurs fois le même groupe");
                           return false;
                       }

       return true;
    }

}



#### 6. Définition des structures de croisement et validation des données

Puis on effectue l'ensemble des étapes de calcul du workflow

In [59]:
using XPlot.Plotly;

// Apariement des élèves à partir de l'inscription

var projectEvaluations = new List<ProjectEvaluation>();  

for (int i = 0; i < evaluations.Count; i++)
{
    var projectEvaluation = new ProjectEvaluation(){ProfessorEmail = professorEmail};
    projectEvaluations.Add(projectEvaluation);

    
    var projetEval = evaluations[i];
    var groupProjectEvaluations = projetEval.GroupBy(e => e.Groupe).Select(g => new GroupEvaluation {
        Groupe = g.Key,
        Evaluations = g.ToList(),
        GroupMembers = studentRecords.Where(s => s.Sujets.Contains(g.Key)).ToList(),
        TeacherWeight = teacherWeight
    }).ToList();
    
    projectEvaluation.GroupedEvaluations.AddRange(groupProjectEvaluations);
    
    // Affichage des élèves en anomalie, typiquement des élèves inscrits mais sans note.
    
    projectEvaluation.FilterEvaluations();


   
    // Affichage des groupes et des moyennes

    display($"Moyenne des notes pour le projet {i + 1}: {projectEvaluation.Moyenne}, écart type: {projectEvaluation.EcartType}");

    // Visualisation avec Plotly
    var chart = Chart.Plot(
        new Histogram
        {
            x = groupProjectEvaluations.Select(n => n.Moyenne),
            autobinx = false,
            xbins = new Xbins { start = 0, end = 20, size = 0.5 }
        }
    );
    
    display(chart);

    // Affichage de la moyenne et l'écart type pour chacun des projets
}




Evaluation écartée: Submission#58+EvaluationRecord, Samy (samy) Hadj-said (hadj) a évalué plusieurs fois le même groupe

Evaluation écartée: Submission#58+EvaluationRecord, Juliette (julie) Jin (jin) a évalué plusieurs fois le même groupe

Evaluation écartée: Submission#58+EvaluationRecord, Samy (samy) Hadj-said (hadj) a évalué plusieurs fois le même groupe

Evaluation écartée: Submission#58+EvaluationRecord, Juliette (julie) Jin (jin) a évalué plusieurs fois le même groupe

Evaluation écartée: Submission#58+EvaluationRecord, Samy (samy) Hadj-said (hadj) a évalué plusieurs fois le même groupe

...

Evaluation écartée: Submission#58+EvaluationRecord, Etudiant jason (jason) perez (perez) membre du groupe évalué, 

Evaluation écartée: Submission#58+EvaluationRecord, Etudiant Djuncie (djunc) Lumban (lumba) non inscrit (djunice.lumban@epita.fr)

Evaluation écartée: Submission#58+EvaluationRecord, Etudiant jason (jason) perez (perez) membre du groupe évalué, 

Moyenne des notes pour le projet 1: 16,341263560933907357952234878, écart type: 0,797021705403362

<!DOCTYPE html>

#### 7. Définition des paramètres de rectification

Définition pour chacun des projets de la moyenne et l'écart type à appliquer

In [63]:
//Déclaration des variables ou utilisation d'une saisie sous forme de boucle.

// Dans un premier temps, une valeur pour tous les projets
List<(decimal newMean, decimal newStdev)> rectificationParams = new List<(decimal newMean, decimal newStdev)>
{
    (15m, 1.5m)
};



#### 8. Application des paramètres de rectification 

Calcul et assignation des notes rectifiées 

In [64]:
using System;
using ClosedXML.Excel;

public static decimal AdjustGrade(decimal originalGrade, decimal groupMean, 
    decimal groupStdDev, decimal targetMean = 10, decimal targetStdDev = 2)
{
    if (groupStdDev == 0) // Pour éviter la division par zéro
        return targetMean; // Retourne la moyenne cible si l'écart-type est nul

    // Calcul de la note ajustée
    decimal adjustedGrade = ((originalGrade - groupMean) / groupStdDev) * targetStdDev + targetMean;

    // Clamper la note pour qu'elle reste dans les bornes 0 à 20
    adjustedGrade = Math.Max(0, Math.Min(20, adjustedGrade));

    return adjustedGrade;
}


// Rectification des notes de chaque groupe pour chaque projet.


for (int i = 0; i < projectEvaluations.Count; i++)
{
    var project = projectEvaluations[i];
    var rectification = rectificationParams[i];
    
    for (int j = 0; j < project.GroupedEvaluations.Count; j++)
    {
        var eval = project.GroupedEvaluations[j];
        decimal originalGrade = eval.Moyenne; // Assurez-vous que la conversion est correcte
        decimal adjustedGrade = eval.NoteRectifiée = AdjustGrade(originalGrade, project.Moyenne, project.EcartType, rectification.newMean, rectification.newStdev);
        
    }
}


### 9. Génération du spreadsheet de résultats

In [65]:
using ClosedXML.Excel;
using System;
using System.Linq;

public string WrapText(string text, int maxLength)
{
    var words = text.Split(' ');
    var wrappedText = new StringBuilder();
    string line = "";

    foreach (var word in words)
    {
        if ((line + word).Length > maxLength)
        {
            wrappedText.AppendLine(line.Trim());
            line = "";
        }
        line += word + " ";
    }

    if (line.Length > 0)
        wrappedText.AppendLine(line.Trim());

    return wrappedText.ToString();
}


public void GenerateWorkbook(List<ProjectEvaluation> projectEvaluations, List<StudentRecord> students)
{
    var workbook = new XLWorkbook();
    var summarySheet = workbook.AddWorksheet("Résumé Étudiants");

    // En-tête pour la feuille de résumé
    summarySheet.Cell(1, 1).Value = "Nom";
    summarySheet.Cell(1, 2).Value = "Prénom";
    int currentColumn = 3;
    for (int i = 0; i < projectEvaluations.Count; i++)
    {
        summarySheet.Cell(1, currentColumn).Value = $"Groupe Projet {i + 1}";
        summarySheet.Cell(1, currentColumn + 1).Value = $"Note Projet {i + 1}";
        currentColumn += 2;
    }
    summarySheet.Cell(1, currentColumn).Value = "Note Moyenne Finale";

    // Trier les étudiants par nom et prénom
    var sortedStudents = students.OrderBy(s => s.Nom).ThenBy(s => s.Prénom).ToList();
    int row = 2;
    foreach (var student in sortedStudents)
    {
        decimal totalNotes = 0;
        int numberOfGrades = 0;
        summarySheet.Cell(row, 1).Value = student.Nom;
        summarySheet.Cell(row, 2).Value = student.Prénom;

        currentColumn = 3;
        for (int i = 0; i < projectEvaluations.Count; i++)
        {
            var project = projectEvaluations[i];
            var eval = project.GroupedEvaluations.FirstOrDefault(g => g.GroupMembers.Contains(student));
            if (eval != null)
            {
                summarySheet.Cell(row, currentColumn).Value = eval.Groupe;
                summarySheet.Cell(row, currentColumn + 1).Value = eval.NoteRectifiée;
                summarySheet.Cell(row, currentColumn + 1).Style.NumberFormat.Format = "0.0";
                totalNotes += eval.NoteRectifiée;
                numberOfGrades++;
            }
            currentColumn += 2;
        }

        decimal finalAverage = numberOfGrades > 0 ? totalNotes / numberOfGrades : 0;
        summarySheet.Cell(row, currentColumn).Value = finalAverage;
        summarySheet.Cell(row, currentColumn).Style.NumberFormat.Format = "0.0";
        row++;
    }

     // Format de tableau
    var table = summarySheet.Range(1, 1, row - 1, currentColumn).CreateTable("SummaryTable");
    // summarySheet.Tables.Add(table);
    table.ShowTotalsRow = false;
    table.Theme = XLTableTheme.TableStyleMedium15;

    // Ajustement des colonnes
    summarySheet.Columns().AdjustToContents();



    // Feuilles de retour qualitatif pour chaque projet
    for (int i = 0; i < projectEvaluations.Count; i++)
    {
        var feedbackSheet = workbook.AddWorksheet($"Projet {i + 1} Feedback");
        feedbackSheet.Cell(1, 1).Value = "Groupe";
        feedbackSheet.Cell(1, 2).Value = "Points Positifs";
        feedbackSheet.Cell(1, 3).Value = "Points Négatifs";
        feedbackSheet.Cell(1, 4).Value = "Recommandations";

        var feedbacks = projectEvaluations[i].GroupedEvaluations;
        int feedbackRow = 2;
        foreach (var feedback in feedbacks)
        {
            foreach (var eval in feedback.Evaluations)
            {
                if (!string.IsNullOrWhiteSpace(eval.PointsPositifs) || !string.IsNullOrWhiteSpace(eval.PointsNégatifs) || !string.IsNullOrWhiteSpace(eval.Recommandations))
                {
                    feedbackSheet.Cell(feedbackRow, 1).Value = feedback.Groupe;
                    feedbackSheet.Cell(feedbackRow, 2).Value = WrapText(eval.PointsPositifs, 40);
                    feedbackSheet.Cell(feedbackRow, 3).Value = WrapText(eval.PointsNégatifs, 40);
                    feedbackSheet.Cell(feedbackRow, 4).Value = WrapText(eval.Recommandations, 40);
                    feedbackRow++;
                }
            }
        }

        // Format de tableau pour les feedbacks
        if (feedbackRow > 2) // S'il y a des données
        {
            var feedbackTable = feedbackSheet.Range(1, 1, feedbackRow - 1, 5).AsTable($"FeedbackTable{i + 1}");
            // feedbackSheet.Tables.Add(feedbackTable);
            feedbackTable.ShowTotalsRow = false;
            feedbackTable.Theme = XLTableTheme.TableStyleLight9;
        }

        // Ajustement automatique des colonnes
        feedbackSheet.Columns().AdjustToContents();

    }


    var savePath  = System.IO.Path.Combine(new System.IO.FileInfo(studentFileInput).DirectoryName, "Resultats_Evaluations.xlsx");
    workbook.SaveAs(savePath);
    display($"Fichier Excel généré : {savePath}");
}

GenerateWorkbook(projectEvaluations, studentRecords);


Fichier Excel généré : g:\Mon Drive\MyIA\Formation\Epita\2024\Resultats_Evaluations.xlsx